# 第二十讲：范数/模（norm）及其应用
___

过拟合（overfitting）是机器学习中比较频繁出现的问题，而规则化（regularization）则是解决过拟合的利器。要使用规则化，我们首先得了解什么是$L_0$、$L_1$、$L_2$以及核范数（nuclear norm）规则化。

监督的机器学习问题无非就是：

> minimizes the error while regularizing the parameters.

也就是在最小化误差的同时正则化参数。最小化误差的目的是为了让模型与训练数据（training data）更好的拟合，而正则化参数的目的则是为了降低模型的复杂度，避免出现过拟合。

增加参数会增加模型的复杂度，一方面可以降低训练误差（training error），但降低训练误差并不是我们的终极目的，我们的真实目的其实是希望模型的测试误差（test error）足够小，也就是能够准确的预测新的数据。因此，我们需要控制模型的复杂度的基础上最小化训练误差，这样得到的模型才能有更佳的泛化性能。而模型的复杂程度就是通过正则化项来控制的。

此外，加入正则项还可以有效约束模型的某些特性，将我们对模型的先验知识融入模型的训练当中，如模型的稀疏（sparse）、平滑（smooth）、低秩（low-rank）等特性。

我们还可以用奥卡姆剃刀原则（Occam's razor）解释正则化：在所有可能的模型中，我们应当选择能很好的解释数据，同时模型复杂度相对较低的模型；从贝叶斯估计的角度来说，正则化项对应于模型的先验概率。此外，还有人认为，正则化是结构风险（structural risk）最小化策略的实现，也就是经验风险加上一个正则化项（regularizer）或惩罚项（penalty term）。

一般来说，监督学习（supervised learning）常常可以表示为最小化形如下面的目标函数：
$$
w^* = \operatorname{arg min}_w \sum_i L(y_i, f(x_i; w)) + \lambda \Omega(w)
$$

其中$L(y_i, f(x_i;w))$是衡量模型（分类或回归）对第$i$个样本的预测值$f(x_i;w)$与观察值$y_i$之间的误差，最小化误差也就是要最大化拟合模型与训练样本。但注意的是，我们不但要保证训练误差足够小，还希望测试误差小，这就是为什么我们加入对参数$w$的正则化函数$\Omega(w)$的目的，也就是要控制模型的复杂度。

当我们将了解了监督学习以后，回过头来，我们就会发现绝大部分的模型都与这个目标函数形神皆似，无非就是变换Loss函数或者正则化函数而已。

例如，改变采用Square loss，则就是最小二乘法（least squares）；如果用Hinge loss， 则变为著名的SVM；如果是log-loss，则模型就是Logistic regression；采用Exp-Loss，则变为大名鼎鼎的Bossting。

例如，对线性回归模型，如果采用$L_2$ penalty，则为ridge regression；如果采用$L_1$ penalty，则为LASSO；如果两者皆用，则为elastic net regression。

## 1. 正则化函数

正则化函数$\Omega(w)$有许多选择，一般来说是模型复杂度的单调递增函数（monotonely increasing function），也即是说，模型越复杂，其值越大。例如，正则化项可以是模型参数的范数。然而，不同的范数对模型参数$w$的约束也不尽相同，取得的效果也大相径庭。

绝大部分的研究论文都集中在这几个范数上：零范数（$L_0$-norm）、一范数（$L_1$-norm）、二范数（$L_2$-norm）、迹范数（trace norm）、F范数（Frobenius norm）、核范数（nuclear norm）。那么这些范数分别都有哪些特性，它们都是如何控制模型特性的呢？且听我一一道来：

## 2. $L_0$范数和$L_1$范数

$L_0$范数的表示方法是：$\lVert \mathbf{w} \rVert_0 = \sum_i \mathbf{1}(w_i \neq 0)$，而$L_1$范数为：$\lVert \mathbf{w} \rVert_1 = \sum_i \lvert w_i \rvert$。

$L_0$范数就是向量或矩阵中非零元素的个数，从直观上来说，最小化参数中非零元素的个数，能直接达到参数选择、以及降低模型复杂度的目的。换言之，也就是希望模型是系数的（sparse）。然而，在铺天盖地的压缩感知（compressed sensing）、稀疏编码（sparse encoding）论文的世界里，稀疏基本上都是通过$L_1$范数实现的，并没有用到$L_0$范数。那么，$L_1$范数是什么？为什么通过最小化$L_1$，可以达到稀疏的目的呢？

$L_1$范数是指向量中各个元素的绝对值之和，也有个别名为“稀疏正则化算子（sparse regularizer）”，是$L_0$范数的最优凸近似。你问我什么是凸？自己回去看一下什么是凸函数哈。那还是没说它为什么能实现稀疏呢？

> 任何的正则化算子，如果在$w_i=0$时候不可微，并且可以分解为“求和”的形式，则该算子能实现稀疏。

而$L_1(W) = \sum_i \lvert w_i \rvert$，$W$在$w_i=0$处不可微，所以能实现稀疏。我们这里只是借用了一下结论，事实上是可以证明的，不过证明的地方你可以自己去查一下啦。

那么，既然$L_0$范数就可以实现稀疏，为什么舍近求远去拜托$L_1$范数呢？我的理解是$L_0$范数的优化求解是NP-hard问题，而$L_1$范数是$L_0$范数的最优凸近似，而且它相比$L_0$范数来说，优化求解要容易的多。这就是为什么$L_1$范数集万千宠爱在一身的原因了。

> 一句话，$L_0$范数和$L_1$范数可以实现稀疏，$L_1$范数因其有更好的优化求解特性而得到更广泛的应用。

那么，实现稀疏的目的是什么？无非两点：实现模型的__特征选择（feature selection）__以及__可解释性（interpretability）__。

## 3. $L_2$范数（$L_2$-norm）

$L_2$范数相比其大兄弟$L_1$-norm来说，其江湖地位也是不惶多让，比如蜚声宇内的岭回归（ridge regression），$L_2$范数就因其“权值衰减（weight decay）”效应而声名大振。那么，$L_2$范数是什么东东？
$$
\lVert \mathbf{w} \rVert = \sqrt{\sum_i w_i^2}
$$

这会使得参数$\mathbf{w}$的分量都尽量小，接近于0，但不会等于0。当参数足够小的时候，模型的复杂度也就相对会更低，也就越不容易产生过拟合现象，这是我们约束$L_2$范数的目的。但注意，$L_2$范数不能进行特征选择。

## 4. 核范数（Nuclear norm）

核范数是仅对矩阵来说的，对大部分人来说比较陌生，在数值上相当于矩阵奇异值（singular values）的和。那么，其作用是什么呢？答案也很简单，保证矩阵的低秩约束（low-rank constraint），那么什么是矩阵的秩（rank），如果你不知道的话，建议回去线性代数重新回炉一番。

对于矩阵$\mathbf{A} \in \mathbb{R}^{m \times n}$来说，如果其秩$\operatorname{rank}(\mathbf{A}) << \min(m, n)$，则我们称矩阵$\mathbf{A}$是低秩的。低秩意味着矩阵具有很强的内在相关性，也就是说矩阵实际可以投影到更低维的线性子空间，也就是用$k<\min(m,n)$个向量就可以完全表达了。

低秩矩阵每行或每列都可以表示为其他的行或列的线性组合，可见它包含大量的冗余信息。利用这种冗余信息，可以进行__缺失数据恢复__，也可以对数据进行__特征提取__。

那么说了半天了，好像没核范数什么事啊？如同$L_0$范数，$\operatorname{rank}(\cdot)$也是非凸的，没错，核范数$\lVert \mathbf{W} \rVert_*$如同$L_1$范数一样，是$\operatorname{rank}(\mathbf{W})$的最优凸近似。

## 5. 范数的应用

说了这么多，似乎我们也应该要说说范数的应用了吧！举几个典型的例子：

### 5.1 矩阵填充（matrix completion）

矩阵填充的一个主流应用是在推荐系统（recommendation system）里面。我们知道，推荐系统有一种方法是通过分析用户的历史记录来给用户推荐的。例如我们在看一部电影的时候，如果喜欢看，就会给它打个分，例如3颗星。然后系统，例如Netflix等知名网站就会分析这些数据，看看到底每部影片的题材到底是怎样的？针对每个人，喜欢怎样的电影，然后会给对应的用户推荐相似题材的电影。但有一个问题是：我们的网站上面有非常多的用户，也有非常多的影片，不是所有的用户都看过说有的电影，不是所有看过某电影的用户都会给它评分。假设我们用一个“用户-影片”的矩阵来描述这些记录，可以看到，会有很多空白的地方。在分析之前，一般需要先对其进行补全，这就叫矩阵填充。

这就是低秩应用的地方了，也就是低秩矩阵重构问题，它可以用如下的模型表述：

> 给定矩阵$\mathbf{A} \in \mathbb{R}^{m \times n}$，如果其中存在某些缺失值，能否根据已知的元素，将这些元素恢复？

当然，如果没有其他的参考条件，想要确定这些数据很困难。但如果我们已知$\mathbf{A}$的秩$\operatorname{rank}(\mathbf{A})\ll m$且$\operatorname{rank}(\mathbf{A})\ll n$，那么我们可以通过矩阵各行(列)之间的线性相关将丢失的元素求出。你会问，这种假定我们要恢复的矩阵是低秩的，合理吗？实际上是十分合理的，比如一个用户对某电影评分是其他用户对这部电影评分的线性组合。所以，通过低秩重构就可以预测用户对其未评价过的视频的喜好程度，从而达到矩阵填充的目的。

### 5.2 鲁棒主成份分析（Robust principal component analysis, RPCA）

主成分分析（principal component analysis）能有效的找出数据中最“关键"的元素和结构，去除噪音和冗余，将原有的复杂数据降维，揭示隐藏在复杂数据背后的简单结构。从线性代数的角度看，这就是一个“搞基”的过程，其目标就是使用另一组基（basis）去重新描述得到的数据空间。希望在这组新的基下，能尽量揭示原有的数据间的关系。这个维度也就是最重要的“主元"。PCA的目标就是找到这样的“主元”，最大程度的去除冗余和噪音的干扰。

PCA的一个关键假设就是噪声是高斯分布的，这样不会影响到矩阵$\mathbf{X} \in \mathbb{R}^{m \times n}$的低秩属性。但是，一旦噪声不是高斯的（比如存在大的噪声和明显的离群点outliers）的话，矩阵$\mathbf{X}$的低秩性就会被破坏，从而变成满秩的（full-rank），这就需要我们将原矩阵$\mathbf{X}$分解为低秩的矩阵$\mathbf{A}$（由于包含内部结构信息，因而是相关的、低秩的），以及稀疏的矩阵$\mathbf{E}$（噪声是稀疏的），这就转变成鲁棒的主成份分析（Robust PCA）：
$$
\begin{array}{ll}
\min_{\mathbf{A, E}} & \operatorname{rank}(\mathbf{A}) + \lambda \lVert \mathbf{E} \rVert_0\\
\textit{s.t.} & \mathbf{X = A + E}
\end{array}
$$

RPCA本质上也是求解数据在低维空间上的最佳投影问题。

由于$\operatorname{rank}(\cdot)$和$L_0$范数在优化上存在非凸和非光滑特性，所以我们一般将它转换成求解下面这个松弛的凸优化问题：
$$
\begin{array}{ll}
\min_{\mathbf{A, E}} & \lVert \mathbf{A} \rVert_* + \lambda \lVert \mathbf{E} \rVert_1\\
\textit{s.t.} & \mathbf{X = A + E}
\end{array}
$$

这个在图像处理上可用于去除上面的噪声点，至于实用用途，你懂的！

## 6. RPCA问题的求解

对于以上的RPCA问题，需要引入拉格朗日乘子（Lagrangian multiplier）$\mathbf{Y}$用增广拉格朗日乘子法进行求解：
$$
L(\mathbf{A, E, Y}, \mu) = \lVert \mathbf{A} \rVert_* + \lambda \lVert \mathbf{E} \rVert_1 + \langle \mathbf{Y}, \mathbf{X - A - E} \rangle + \frac{\mu}{2} \lVert X - A - E \rVert_F
$$
其中$\mu$是正实数，$\lVert \cdot \rVert$表示的是Frobenius范数，F-范数的定义是：
$$
\lVert \mathbf{A} \rVert_\mathcal{F}^2 = \sum_{i,j}\lvert a_{ij} \rvert^2 = \sum_i \lVert A_{i*}\rVert_2^2 = \sum_j \lVert A_{*j} \rVert_2^2 = \operatorname{trace}(\mathbf{A^TA})
$$

这个优化问题可用inexact拉格朗日乘子法（augmented Lagrangian method of multipliers, ALM）求解，细节请参见文章：

> Lin Z, Chen M, Wu L, Ma Y: [The augmented Lagrange multiplier method for exact recovery of corrupted low-rank matrices](http://Arxivorg/abs/10095055v2), 2010.

## 7. 小结

值得注意的是，向量和矩阵的范数定义存在一些差别，使用的时候要小心。对Python用户来说，`numpy.linalg.norm()`函数可以帮助我们计算向量和矩阵的范数。

### 7.1 矩阵的范数

#### (1) $L_1$范数：列和范数
$$
\lVert \mathbf{A} \rVert_1 = \max_j \sum_i \lvert a_{ij} \rvert
$$

#### (2) $L_2$范数：谱范数
$$
\lVert \mathbf{A} \rVert_2 = \sqrt{\lambda_1}
$$
其中$\lambda_1$为$\mathbf{A^TA}$的最大特征值。

#### (3) Frobenius范数：F-范数
$$
\lVert \mathbf{A} \rVert_F = \sqrt{\sum_i \sum_j \lvert a_{ij} \rvert^2}
$$


#### (4) 无穷范数：行和范数
$$
\lVert \mathbf{A} \rVert_{\infty} = \max_i \sum_j \lvert a_{ij} \rvert
$$


### 7.2 向量的范数

#### $L_p$范数
$$
\lVert \mathbf{w} \rVert_p = \left( \sum_i \lvert \mathbf{w}_i \rvert^p \right)^{1/p}
$$
- 当$p=0$：$\lVert \mathbf{w} \rVert_0 = \sum_i \lvert \mathbf{w}_i \rvert^0$；
- 当$p=1$：$\lVert \mathbf{w} \rVert_1 = \sum_i \lvert \mathbf{w}_i \rvert$；
- 当$p=2$：$\lVert \mathbf{w} \rVert_2 = \sqrt{\sum_i \lvert \mathbf{w}_i \rvert^2}$；
- 当$p=+\infty$：$\lVert \mathbf{w} \rVert_{\infty} = \max_i \lvert \mathbf{w}_i \rvert $；
- 当$p=-\infty$：$\lVert \mathbf{w} \rVert_{-\infty} = \min_i \lvert \mathbf{w}_i \rvert $；

In [1]:
import numpy as np
from numpy.linalg import norm

In [17]:
X = np.random.randn(10,9)
norm(X, ord=1) == abs(X).sum(axis=0).max()

True

## 附录：生物信息学应用

1. [Gene expression prediction using low-rank matrix completion](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-016-1106-6)

2. [Genotype imputation via matrix completion](http://genome.cshlp.org/content/early/2012/12/10/gr.145821.112)

3. [A Singular Value Thresholding Algorithm for Matrix Completion](http://dx.doi.org/10.1137/080738970)

4. [Cortical surface biomarkers for predicting cognitive outcomes using group L2,1 norm](http://dx.doi.org/10.1016/j.neurobiolaging.2014.07.045)